In [1]:
import mujoco
import numpy as np
import time
import tempfile
import os
from pathlib import Path
from mujoco import viewer
import shutil
import re

In [2]:
# Get the absolute path to your FR3 model
current_dir = os.path.abspath(os.getcwd())
fr3_dir = os.path.join(current_dir, "FR3_MuJoCo/franka_fr3")
fr3_xml_path = os.path.join(fr3_dir, "fr3.xml")

In [3]:
model = mujoco.MjModel.from_xml_path(fr3_xml_path)
data = mujoco.MjData(model)

In [4]:
save_dir = "FR3_MuJoCo/franka_fr3"
scene_xml_path = os.path.join(save_dir, "franka_with_table.xml")

In [5]:
fr3_xml_path = os.path.join(fr3_dir, "fr3.xml")
with open(fr3_xml_path, 'r') as f:
    fr3_xml_content = f.read()

# Remove all keyframe definitions
fr3_xml_content = re.sub(r'<keyframe.*?</keyframe>', '', fr3_xml_content, flags=re.DOTALL)

# Modify the XML to add table and box
if '</worldbody>' in fr3_xml_content:
    modified_xml = fr3_xml_content.replace('</worldbody>', '''
    <!-- Table -->
    <body name="table" pos="0.5 0.5 -0.05">
        <geom name="tabletop" type="box" size="0.7 0.7 0.05" rgba="0.8 0.8 0.8 1" mass="10"/>
        <geom name="leg1" type="box" pos="0.55 0.55 -0.4" size="0.025 0.025 0.4" rgba="0.8 0.8 0.8 1" mass="1"/>
        <geom name="leg2" type="box" pos="0.55 -0.55 -0.4" size="0.025 0.025 0.4" rgba="0.8 0.8 0.8 1" mass="1"/>
        <geom name="leg3" type="box" pos="-0.55 0.55 -0.4" size="0.025 0.025 0.4" rgba="0.8 0.8 0.8 1" mass="1"/>
        <geom name="leg4" type="box" pos="-0.55 -0.55 -0.4" size="0.025 0.025 0.4" rgba="0.8 0.8 0.8 1" mass="1"/>
    </body>
    
    <!-- Movable Box - with freejoint for full 6DOF movement (including XYZ translation) -->
    <body name="box" pos="0.5 0 0.05">
        <freejoint name="box_freejoint"/>
        <geom name="box_geom" type="box" size="0.05 0.05 0.05" rgba="0.5 1 0.8 1"/>
        <inertial pos="0 0 0" mass="3" diaginertia="0.001 0.001 0.001"/>
    </body>
</worldbody>''')
    
    save_file_path = os.path.join(save_dir, "fr3_with_moveable_box.xml")
    
    # Save the modified XML permanently
    with open(save_file_path, 'w') as f:
        f.write(modified_xml)
    
    # Test load the saved model
    model = mujoco.MjModel.from_xml_path(save_file_path)
    data = mujoco.MjData(model)
    print("Saved model loads successfully!")


Saved model loads successfully!


In [ ]:
# Create a viewer and visualize the model
viewer.launch(model, data)

/home/kai/.local/lib/python3.10/site-packages/glfw/__init__.py:917: GLFWError: (65548) b'Wayland: The platform does not provide the window position'
  warnings.warn(message, GLFWError)


: 